<a href="https://colab.research.google.com/github/digdemorhan/Keras_With_Examples_For_Deep_Learning/blob/main/KerasCustomLayer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras

***Linear isimli bir class oluşturulmuş ve keras.layers.Layer'dan miras almıştır.***

***init fonksiyonu: Katmanın bileşenleri tanımlanır.***

***call fonksiyonu: Katmanın yapacağı işler belirlenir.***

In [ ]:
class Linear(keras.layers.Layer):
  def __init__(self, units=32, input_dim=32):
    super(Linear, self).__init__()
    #Ağırlıkların oluşturulması
    w_init = tf.random_normal_initializer()
    self.w = tf.Variable(
        initial_value = w_init(shape=(input_dim, units), dtype="float32"),
        trainable = True #Ağırlıklar back-propagation yöntemiyle eğitilebilir
    )
    #Bias oluşturulması
    b_init = tf.zeros_initializer()
    self.b = tf.Variable(
        initial_value = b_init(shape=(units,), dtype="float32"),
        trainable = True
    )
  #Call metodunun oluşturulması
  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

In [ ]:
#1 değerlerini içeren 2x2'lik matris oluşturulması
x = tf.ones((2,2))
print(x)

tf.Tensor(
[[1. 1.]
 [1. 1.]], shape=(2, 2), dtype=float32)


In [ ]:
linear_layer = Linear(4,2) #nöron sayısı 4 ve girdi boyutu 2
#Verinin oluşturulan katmandan geçirilmesi
y = linear_layer(x)
print(y)

tf.Tensor(
[[-0.07939168 -0.07642309  0.10071824  0.09242015]
 [-0.07939168 -0.07642309  0.10071824  0.09242015]], shape=(2, 4), dtype=float32)


In [ ]:
#Katmanda kullanılan ağırlıkların ve bias değerlerinin görülmesi
print(linear_layer.w)
print(linear_layer.b)

<tf.Variable 'Variable:0' shape=(2, 4) dtype=float32, numpy=
array([[-0.07741177, -0.02923107,  0.07154857, -0.00600545],
       [-0.00197991, -0.04719202,  0.02916968,  0.0984256 ]],
      dtype=float32)>
<tf.Variable 'Variable:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>


In [ ]:
from tensorflow.python.ops.init_ops_v2 import Initializer
#add_weight metodu ile ağırlıkların tanımlanması
class Linear(keras.layers.Layer):
  def __init__(self, units=32, input_dim=32):
    super(Linear, self).__init__()
    self.w = self.add_weight(shape=(input_dim, units),
                             initializer = "random_normal",
                             trainable = True)
    self.b = self.add_weight(shape=(units,),
                             initializer = "zeros",
                             trainable = True)
  #Call metodunun oluşturulması
  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

In [ ]:
#Örnek veriler ile katmanın çalıştırılması
x = tf.ones((2,2))
linear_layer = Linear(4,2)
y = linear_layer(x)
print(y)

tf.Tensor(
[[ 0.10139559 -0.03959316 -0.01549093 -0.01182913]
 [ 0.10139559 -0.03959316 -0.01549093 -0.01182913]], shape=(2, 4), dtype=float32)


In [ ]:
#Girdi boyutu parametre olarak verilmeden katman oluşturulması
class Linear(keras.layers.Layer):
  def __init__(self, units=32):
    super(Linear, self).__init__()
    self.units = units
  #Ağırlıkların oluşturulması
  def build(self, input_shape):
      self.w = self.add_weight(shape=(input_shape[-1], self.units),
                               initializer = "random_normal",
                               trainable = True)
      self.b = self.add_weight(shape=(self.units,),
                               initializer = "random_normal",
                               trainable = True)
  #Call metodunun oluşturulması
  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b


In [ ]:
#Katman örneğinin alınması
linear_layer = Linear(32) #sadece nöron değeri alındı, girdi boyutu verilmedi
y = linear_layer(x)
print(y)

tf.Tensor(
[[ 0.07690775 -0.00634513  0.15965573  0.08545148 -0.06408241 -0.11090479
  -0.09871972 -0.09781678 -0.022215   -0.08527908 -0.08241413  0.01207876
   0.13015804  0.02233273  0.0311064  -0.00754188  0.03806047 -0.02302093
  -0.0555428   0.09757417  0.01983105  0.13677038 -0.07714042  0.08077806
   0.10437647 -0.17869686  0.09256818 -0.07262797  0.04237331  0.02696074
   0.08513233  0.00655323]
 [ 0.07690775 -0.00634513  0.15965573  0.08545148 -0.06408241 -0.11090479
  -0.09871972 -0.09781678 -0.022215   -0.08527908 -0.08241413  0.01207876
   0.13015804  0.02233273  0.0311064  -0.00754188  0.03806047 -0.02302093
  -0.0555428   0.09757417  0.01983105  0.13677038 -0.07714042  0.08077806
   0.10437647 -0.17869686  0.09256818 -0.07262797  0.04237331  0.02696074
   0.08513233  0.00655323]], shape=(2, 32), dtype=float32)


In [ ]:
#Art arda gelen katmanların birleştirilmesi
class MLPBlock(keras.layers.Layer):
  def __init__(self):
    super(MLPBlock, self).__init__()
    self.linear_1 = Linear(32)
    self.linear_2 = Linear(32)
    self.linear_3 = Linear(1)
  #Call metodunun oluşturulması
  def call(self, inputs):
    x = self.linear_1(inputs)
    x = tf.nn.relu(x)
    x = self.linear_2(x)
    x = tf.nn.relu(x)
    return self.linear_3(x)

In [ ]:
#MLPBlock sınıfından örnek alınması
mlp = MLPBlock()
y = mlp(tf.ones(shape=(3,64)))


In [ ]:
#Ağırlıkların görüntülenmesi
for weight in mlp.weights:
  print(weight.numpy())

[[-0.02043233  0.03074252  0.01606071 ... -0.01995402  0.0018457
  -0.11062484]
 [ 0.00922695 -0.02579582 -0.00732633 ... -0.04512377 -0.01808306
   0.09827697]
 [ 0.05409205 -0.06161223 -0.06629013 ...  0.00246336 -0.04618117
   0.01151854]
 ...
 [-0.01648197 -0.04132658 -0.02596946 ...  0.00598168  0.02528627
   0.05343216]
 [-0.04229448  0.03447316 -0.03461614 ...  0.07299665 -0.02033533
  -0.0296888 ]
 [ 0.06491765 -0.00724054 -0.02330712 ...  0.11028557  0.078206
  -0.00770357]]
[ 0.02763378 -0.03676413  0.01258137  0.02914702  0.02366941 -0.02128512
  0.03752587 -0.05199597 -0.02023182  0.0155437  -0.01862787 -0.02912346
 -0.06639066 -0.09247016 -0.02395329 -0.071996    0.04897406 -0.0363383
  0.08835216  0.00629579  0.00375932  0.01022203  0.02115201  0.02469125
 -0.03755471  0.04530199  0.11749784 -0.03897502 -0.00917242  0.10950817
  0.06378492 -0.0779451 ]
[[ 0.03607823 -0.01537471  0.00042428 ... -0.12802504 -0.04723136
  -0.04839142]
 [-0.04036984 -0.06282465 -0.04769212 ..